In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

# Step 1: Data Preprocessing
# data = pd.read_excel("NEW DATA SHEET (copy) with features.xlsx")  

data=pd.read_excel("Roulette_data.xlsx")

In [2]:
data.head()

,number,dozen,column,parity,color,series,group
0,6,D1,C3,EVEN,BLACK,B,G2
1,10,D1,C1,EVEN,BLACK,C,G2
2,35,D3,C2,ODD,BLACK,A,G1
3,0,0,0,0,GREEN,A,G1
4,20,D2,C2,EVEN,BLACK,B,G2


In [3]:
# categorical_columns=['Dozen', 'Column', 'odd/even', 'red / black', 'series', "Group"]   
categorical_columns=['dozen', 'column', 'parity', 'color', 'series', 'group']
# Convert all categorical columns to strings to ensure uniformity
data[categorical_columns] = data[categorical_columns].astype(str)

In [4]:
data.dozen.value_counts()

dozen
D1    22014
D2    21870
D3    21641
0      1877
Name: count, dtype: int64

In [5]:
data.series.value_counts()

series
A    30894
C    21981
B    14527
Name: count, dtype: int64

In [6]:
data.group.value_counts()


group
G1    34591
G2    32811
Name: count, dtype: int64

In [7]:
data.columns

Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [ ]:
for i in ['dozen', 'column', 'parity', 'color', 'series', 'group']:
    # print(i)
    print(data[i].value_counts())
    print("\n")

dozen
D1    22014
D2    21870
D3    21641
0      1877
Name: count, dtype: int64


column
C2    22025
C3    21781
C1    21719
0      1877
Name: count, dtype: int64


parity
EVEN    32959
ODD     32566
0        1877
Name: count, dtype: int64


color
BLACK    32804
RED      32721
GREEN     1877
Name: count, dtype: int64


series
A    30894
C    21981
B    14527
Name: count, dtype: int64


group
G1    34591
G2    32811
Name: count, dtype: int64




In [9]:
# data.Dozen.replace({"D 1":"D1", "D 2":"D2", "D 3":"D3"}, inplace=True)
# data.Column.replace({"C 1":"C1", "C 2":"C2", "C 3":"C3"}, inplace=True)
# data["odd/even"].replace({"nan":"0"}, inplace=True)


In [10]:
data.dozen.value_counts()

dozen
D1    22014
D2    21870
D3    21641
0      1877
Name: count, dtype: int64

In [11]:
data.column.value_counts()

column
C2    22025
C3    21781
C1    21719
0      1877
Name: count, dtype: int64

In [12]:
# data.Column.replace({"C 1":"C1", "C 2":"C2", "C 3":"C3"}, inplace=True)

In [13]:
data["parity"].value_counts()

parity
EVEN    32959
ODD     32566
0        1877
Name: count, dtype: int64

In [14]:
data[data["parity"]=="0"]

,number,dozen,column,parity,color,series,group
3,0,0,0,0,GREEN,A,G1
9,0,0,0,0,GREEN,A,G1
43,0,0,0,0,GREEN,A,G1
182,0,0,0,0,GREEN,A,G1
433,0,0,0,0,GREEN,A,G1
...,...,...,...,...,...,...,...
67165,0,0,0,0,GREEN,A,G1
67262,0,0,0,0,GREEN,A,G1
67331,0,0,0,0,GREEN,A,G1
67347,0,0,0,0,GREEN,A,G1


In [15]:
# data["odd/even"].replace({"nan":0}, inplace=True)

In [16]:
data["series"].value_counts()

series
A    30894
C    21981
B    14527
Name: count, dtype: int64

In [17]:
data["color"].value_counts()

color
BLACK    32804
RED      32721
GREEN     1877
Name: count, dtype: int64

In [18]:
data["group"].value_counts()

group
G1    34591
G2    32811
Name: count, dtype: int64

In [19]:
data["dozen"].value_counts()

dozen
D1    22014
D2    21870
D3    21641
0      1877
Name: count, dtype: int64

In [20]:
data["column"].value_counts()

column
C2    22025
C3    21781
C1    21719
0      1877
Name: count, dtype: int64

In [21]:
data["parity"].value_counts()

parity
EVEN    32959
ODD     32566
0        1877
Name: count, dtype: int64

In [22]:
data.columns

Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [23]:
# for i in ['Dozen', 'Column', 'odd/even', 'red / black', 'series',
#        'Group']:
#     print(i, data[i].value_counts())

In [24]:
model_data=data.copy()

In [25]:
model_data.head()

,number,dozen,column,parity,color,series,group
0,6,D1,C3,EVEN,BLACK,B,G2
1,10,D1,C1,EVEN,BLACK,C,G2
2,35,D3,C2,ODD,BLACK,A,G1
3,0,0,0,0,GREEN,A,G1
4,20,D2,C2,EVEN,BLACK,B,G2


In [26]:
model_data.columns


Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [27]:
# model_data[model_data["Dozen"]=="0"]

In [28]:
#Remove rows with 0 in Dozen, Column, odd/even, red/black, series, Group
# model_data=model_data[model_data["Dozen"]!="0"]
# model_data[model_data["Column"]=="0"]

In [29]:
for i in ['dozen', 'column', 'parity', 'color', 'series', 'group']:
    print(i, model_data[i].value_counts())

dozen dozen
D1    22014
D2    21870
D3    21641
0      1877
Name: count, dtype: int64
column column
C2    22025
C3    21781
C1    21719
0      1877
Name: count, dtype: int64
parity parity
EVEN    32959
ODD     32566
0        1877
Name: count, dtype: int64
color color
BLACK    32804
RED      32721
GREEN     1877
Name: count, dtype: int64
series series
A    30894
C    21981
B    14527
Name: count, dtype: int64
group group
G1    34591
G2    32811
Name: count, dtype: int64


In [30]:
model_data.columns

Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [31]:
model_data.head()

,number,dozen,column,parity,color,series,group
0,6,D1,C3,EVEN,BLACK,B,G2
1,10,D1,C1,EVEN,BLACK,C,G2
2,35,D3,C2,ODD,BLACK,A,G1
3,0,0,0,0,GREEN,A,G1
4,20,D2,C2,EVEN,BLACK,B,G2


In [32]:
# for i in ['Dozen', 'Column', 'odd/even', 'red / black', 'series',
#        'Group']:
#     print(i, model_data[i].value_counts())

In [33]:
model_data.dtypes

number     int64
dozen     object
column    object
parity    object
color     object
series    object
group     object
dtype: object

### Prediction on Series 

In [34]:
series_data=model_data.copy()

In [35]:
categorical_columns=['dozen', 'column', 'parity', 'color', 'series', 'group']

label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    series_data[column] = le.fit_transform(series_data[column])
    label_encoders[column] = le

In [36]:
series_data.head()

,number,dozen,column,parity,color,series,group
0,6,1,3,1,0,1,1
1,10,1,1,1,0,2,1
2,35,3,2,2,0,0,0
3,0,0,0,0,1,0,0
4,20,2,2,1,0,1,1


In [37]:
for i in categorical_columns:
    print(i, series_data[i].value_counts())

dozen dozen
1    22014
2    21870
3    21641
0     1877
Name: count, dtype: int64
column column
2    22025
3    21781
1    21719
0     1877
Name: count, dtype: int64
parity parity
1    32959
2    32566
0     1877
Name: count, dtype: int64
color color
0    32804
2    32721
1     1877
Name: count, dtype: int64
series series
0    30894
2    21981
1    14527
Name: count, dtype: int64
group group
0    34591
1    32811
Name: count, dtype: int64


In [38]:
series_data=series_data.drop(["number"], axis=1)

In [39]:
series_data.head()

,dozen,column,parity,color,series,group
0,1,3,1,0,1,1
1,1,1,1,0,2,1
2,3,2,2,0,0,0
3,0,0,0,1,0,0
4,2,2,1,0,1,1


In [40]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

def preprocess_data(data):
    # Check for columns with non-numeric data
    print("Checking data types before preprocessing:")
    print(data.dtypes)

    # Normalize 'Dozen', "series" and 'Column' values (if they are now numeric)
    scaler = StandardScaler()
    data[['dozen', 'column', 'parity', 'color','group']] = scaler.fit_transform(data[['dozen', 'column', 'parity', 'color', 'group']])

    # Fill missing values if necessary
    data = data.fillna(0)
    
    return data

# data = preprocess_data(data)



In [41]:
data=preprocess_data(series_data)

Checking data types before preprocessing:
dozen     int64
column    int64
parity    int64
color     int64
series    int64
group     int64
dtype: object


In [42]:
data.head() 

,dozen,column,parity,color,series,group
0,-1.080172,1.214899,-0.826206,-1.012974,1,1.026767
1,-1.080172,-1.088716,-0.826206,-1.012974,2,1.026767
2,1.221077,0.063091,0.988384,-1.012974,0,-0.973931
3,-2.230796,-2.240524,-2.640796,0.001249,0,-0.973931
4,0.070452,0.063091,-0.826206,-1.012974,1,1.026767


In [43]:
# # Step 2: Prepare features (X) and target (y)
# # We will predict 'series' for the next round

# X = []
# y = []

# sequence_length = 5  # Number of previous rounds to consider as input
# for i in range(len(data) - sequence_length):
#     # print("-------------------")
#     # print(i+sequence_length)
#     # print(data.iloc[i:i+sequence_length].drop(columns=['Group']).values)
#     X.append(data.iloc[i:i+sequence_length].drop(columns=['Group']).values)# Features (all columns except 'series')
    
#     # print(data.iloc[i+sequence_length]['Group'])
#     # print("________________")
#     y.append(data.iloc[i+sequence_length]['Group'])  # Target (next round's 'series')

# X = np.array(X)
# y = np.array(y)

# # Reshape X for LSTM [samples, time steps, features]
# X = X.reshape(X.shape[0], X.shape[1], X.shape[2])




In [44]:
labels = data['series'].values

In [45]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [46]:
data.head()

,dozen,column,parity,color,series,group
0,-1.080172,1.214899,-0.826206,-1.012974,1,1.026767
1,-1.080172,-1.088716,-0.826206,-1.012974,2,1.026767
2,1.221077,0.063091,0.988384,-1.012974,0,-0.973931
3,-2.230796,-2.240524,-2.640796,0.001249,0,-0.973931
4,0.070452,0.063091,-0.826206,-1.012974,1,1.026767


In [47]:
# Step 2: Prepare features (X) and target (y)
X = []
y = []
time_steps = 10
num_features=list(np.unique(labels))
# sequence_length = 5  # Number of previous rounds to consider as input
for i in range(len(data) - time_steps):
    # print("-------------------")

    X.append(data.iloc[i:i+time_steps].drop(columns=['series']).values)# Features (all columns except 'series')
    y.append(data.iloc[i+time_steps]['series'])  # Target (next round's 'series')

X = np.array(X)
y = np.array(y)
# Reshape X for LSTM [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])
num_features = X.shape[-1]


In [48]:

# # Step 2: Prepare features (X) and target (y)
# # We will predict 'series' for the next round

# X = []
# y = []
# time_steps = 50
# num_features=list(np.unique(labels))
# # sequence_length = 5  # Number of previous rounds to consider as input
# for i in range(len(data) - time_steps):
#     # print("-------------------")
#     # print(data.iloc[i:i+sequence_length].drop(columns=['Group']).values)
#     X.append(data.iloc[i:i+time_steps].drop(columns=['Group']).values)# Features (all columns except 'series')
#     y.append(data.iloc[i+time_steps]['Group'])  # Target (next round's 'series')

# X = np.array(X)
# y = np.array(y)

# # Reshape X for LSTM [samples, time steps, features]
# X = X.reshape(X.shape[0], X.shape[1], X.shape[2])
# num_features = X.shape[-1]

In [49]:
data["series"].value_counts()

series
0    30894
2    21981
1    14527
Name: count, dtype: int64

In [50]:
num_features

5

In [51]:
# Train-Test Split (No separate validation set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)


In [52]:
print(X_train.shape)  # Expected: (number_of_samples, time_steps, num_features)


(66718, 10, 5)


In [53]:
X_train=X
y_train=y

In [54]:
# # Build the LSTM Model
# model = Sequential([
#     LSTM(64, activation='tanh', return_sequences=True, input_shape=(time_steps, X_train.shape[2])),
#     Dropout(0.2),
#     LSTM(32, activation='tanh'),
#     Dropout(0.2),
#     Dense(32, activation='relu'),
#     Dense(3, activation='softmax')  # Use 4 units for 4 classes (0, 1, 2, 3)
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # ================================
# # Step 3: Train the Model
# # ================================
# # Callbacks for early stopping and saving the best model
# early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('best_model_odd_even.keras', monitor='loss', save_best_only=True)

# # Train the model
# history = model.fit(
#     X_train, y_train,
#     epochs=50,
#     batch_size=32,
#     callbacks=[early_stopping, model_checkpoint]
# )

# # Evaluate the model
# accuracy = model.evaluate(X_test, y_test)
# print(f"Test Accuracy: {accuracy[1]*100:.2f}%")


In [55]:
data["series"].value_counts()

series
0    30894
2    21981
1    14527
Name: count, dtype: int64

In [56]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler



# # ================================
# # Step 2: Build the LSTM Model
# # ================================
# num_classes = 3  # Number of classes
# time_steps = X_train.shape[1]
# num_features = X_train.shape[2]

# model = Sequential([
#     LSTM(32, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(time_steps, num_features)),
#     Dropout(0.3),
#     Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
#     Dense(num_classes, activation='softmax')  # Use softmax for multi-class classification
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # ================================
# # Step 3: Train the Model
# # ================================
# # Callbacks for early stopping, model checkpoint, and learning rate reduction
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
# lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

# # Train the model
# history = model.fit(
#     X_train, y_train,
#     validation_split=0.2,
#     epochs=50,
#     batch_size=32,
#     callbacks=[early_stopping, model_checkpoint, lr_scheduler]
# )



In [57]:
# ================================
num_classes = 3  # Number of classes
time_steps = 10
num_features = X_train.shape[2]

# Build the LSTM Model
model = Sequential([
    LSTM(64, activation='tanh', return_sequences=True, input_shape=(time_steps, num_features)),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')  # Use 4 units for 4 classes (0, 1, 2, 3)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ================================
# Step 3: Train the Model
# ================================
# Callbacks for early stopping and saving the best model
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_odd_even.keras', monitor='loss', save_best_only=True)
#lr=0.01
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy[1]*100:.2f}%")


Epoch 1/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.4560 - loss: 1.0595
Epoch 2/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4590 - loss: 1.0536
Epoch 3/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4576 - loss: 1.0537
Epoch 4/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4598 - loss: 1.0527
Epoch 5/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4614 - loss: 1.0525
Epoch 6/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4622 - loss: 1.0517
Epoch 7/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4582 - loss: 1.0549
Epoch 8/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4569 - loss: 1.0542
Epoch 9/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4627 - loss: 1.0515
Epoch 10/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4599 - loss: 1.0530
Epoch 11/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4595 - loss: 1.0531
Epoch 12/50
1053/1053 ━━━━━━━━

In [59]:
# ================================
# Step 4: Evaluate the Model
# ================================
accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy[1] * 100:.2f}%")

# # ================================
# # Step 5: Visualize Training Results
# # ================================
# import matplotlib.pyplot as plt

# # Plot training and validation loss
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['test_loss'], label='Test Loss')
# plt.legend()
# plt.title('Loss')
# plt.show()

# # Plot training and validation accuracy
# plt.plot(history.history['accuracy'], label='Train Accuracy')
# plt.plot(history.history['test_accuracy'], label='Test Accuracy')
# plt.legend()
# plt.title('Accuracy')
# plt.show()


Test Accuracy: 59.20%


In [60]:
label_encoders

{'dozen': LabelEncoder(),
 'column': LabelEncoder(),
 'parity': LabelEncoder(),
 'color': LabelEncoder(),
 'series': LabelEncoder(),
 'group': LabelEncoder()}

In [61]:
# Get the single predicted class
sample_prediction=model.predict(X_test[6].reshape(1, time_steps, num_features))
sample_label = np.argmax(sample_prediction)
sample_pred=label_encoders["series"].inverse_transform([sample_label])

print(f"\nPredicted Label for Sample Input: {sample_label} , {sample_pred}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step

Predicted Label for Sample Input: 0 , ['A']


In [62]:
X_test.shape

(674, 10, 5)

In [63]:
# Get the single predicted class
sample_prediction=model.predict(X_test[6].reshape(1, time_steps, num_features))
sample_label = np.argmax(sample_prediction, axis=1)
sample_pred=label_encoders["series"].inverse_transform([sample_label])

print(f"\nPredicted Label for Sample Input: {sample_label} , {sample_pred}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

Predicted Label for Sample Input: [0] , ['A']


In [68]:
#Model 
model.save("Series_lstm_model_10.h5")

In [69]:
from tensorflow.keras.models import load_model
model1=load_model("Series_lstm_model_10.h5")

In [65]:
y_pred=model.predict(X_test)
y_pred=np.argmax(y_pred, axis=1)
predictions=label_encoders["series"].inverse_transform(y_pred)


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


In [68]:
from sklearn.metrics import accuracy_score
test_acc=accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_acc*100:.2f}%")

Test Accuracy: 59.20%


In [73]:
df=pd.DataFrame({"Actual":y_test, "Predicted":y_pred, "Predictions":predictions})
df[df["Actual"]==df["Predicted"]]

,Actual,Predicted,Predictions
0,0.0,0,A
1,0.0,0,A
3,1.0,1,B
4,0.0,0,A
7,1.0,1,B
...,...,...,...
666,0.0,0,A
668,0.0,0,A
671,0.0,0,A
672,0.0,0,A


In [74]:
class_report=classification_report(y_test, y_pred)  
print(class_report)

              precision    recall  f1-score   support

         0.0       0.57      0.88      0.69       305
         1.0       0.65      0.35      0.45       151
         2.0       0.63      0.36      0.46       218

    accuracy                           0.59       674
   macro avg       0.62      0.53      0.53       674
weighted avg       0.61      0.59      0.56       674



In [66]:
predictions

array(['A', 'A', 'A', 'B', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B',
       'A', 'A', 'A', 'C', 'A', 'A', 'C', 'C', 'A', 'A', 'C', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'C', 'A', 'B', 'A', 'C', 'C', 'C',
       'A', 'A', 'C', 'C', 'A', 'A', 'A', 'C', 'C', 'C', 'B', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'C', 'C', 'A', 'A', 'A', 'A', 'B', 'A',
       'C', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'C', 'A', 'B', 'A', 'C',
       'B', 'A', 'C', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'B', 'A', 'A', 'A', 'B', 'A', 'A', 'C', 'A', 'B', 'A', 'A',
       'A', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'C', 'A', 'C', 'A',
       'A', 'C', 'A', 'C', 'A', 'B', 'A', 'A', 'A', 'C', 'C', 'B', 'A',
       'A', 'A', 'C', 'B', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'B', 'C',
       'A', 'A', 'A', 'B', 'A', 'A', 'C', 'B', 'A', 'A', 'A', 'B', 'A',
       'A', 'A', 'A', 'C', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A

In [84]:
# # ================================
# # Step 4: Evaluate the Model
# # ================================
# # Evaluate on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
# print(f"Test Loss: {test_loss:.4f}")
# print(f"Test Accuracy: {test_accuracy:.4f}")

# # Predict on the test set
# y_pred_prob = model.predict(X_test)
# y_pred = (y_pred_prob > 0.5).astype(int)

# # Confusion Matrix
# conf_matrix = confusion_matrix(y_test, y_pred)
# plt.figure(figsize=(8, 6))

In [85]:
X_test.shape
X_test[:1].shape

(1, 50, 5)

In [86]:
X_test[:1]

array([[[-1.08193769e+00,  6.20481932e-02, -9.37440845e-01,
         -1.01310394e+00,  1.02729149e+00],
        [ 6.91113935e-02,  1.21453727e+00, -9.37440845e-01,
         -1.01310394e+00,  1.02729149e+00],
        [-1.08193769e+00, -1.09044089e+00, -9.37440845e-01,
         -1.01310394e+00, -9.73433549e-01],
        [ 6.91113935e-02,  6.20481932e-02, -9.37440845e-01,
         -1.01310394e+00,  1.02729149e+00],
        [-2.23298677e+00, -2.24292996e+00, -9.37440845e-01,
          1.03529102e-03, -9.73433549e-01],
        [-1.08193769e+00, -1.09044089e+00, -9.37440845e-01,
         -1.01310394e+00,  1.02729149e+00],
        [ 6.91113935e-02,  6.20481932e-02,  1.02245846e+00,
         -1.01310394e+00, -9.73433549e-01],
        [ 6.91113935e-02,  1.21453727e+00,  1.02245846e+00,
         -1.01310394e+00, -9.73433549e-01],
        [ 6.91113935e-02,  6.20481932e-02, -9.37440845e-01,
          1.01517452e+00,  1.02729149e+00],
        [ 1.22016048e+00, -1.09044089e+00,  1.02245846e+00,
    

In [87]:
# #Predict on the test set
# y_pred_prob = model.predict(X_test[:1])
# y_pred = (y_pred_prob > 0.5).astype(int)
# print(y_pred)
# y_pred=label_encoders["odd/even"].inverse_transform(y_pred)

# # # Save the model
# model.save("lstm_model_ODD_EVEN.h5")

In [75]:
model.save("6_Series_lstm_model_10.h5")

### Inference Script


In [88]:
data2=pd.read_excel("NEW DATA SHEET (copy) with features.xlsx") 

In [89]:
# from tensorflow.keras.models import load_model

# # Load the trained model
# model_path = "best_model2.keras"
# model = load_model(model_path)
# print("Model loaded successfully.")

In [90]:
model.predict(X_test[:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


array([[0.46142247, 0.21467936, 0.32389817]], dtype=float32)

In [91]:
data2[:1]


,Number,Dozen,Column,odd/even,red / black,series,Group
0,6,D 1,C 3,EVEN,BLACK,B,G2


In [92]:
import warnings
warnings.filterwarnings("ignore")   

In [104]:
import numpy as np
from tensorflow.keras.models import load_model

# ================================
# Step 1: Load the Trained Model
# ================================
def load_trained_model(model_path):
    """Load the trained model from a file."""
    model = load_model(model_path)
    print("Model loaded successfully.")
    return model

# ================================
# Step 2: Preprocess Input Data
# ================================
def preprocess_input(input_data):
    """Preprocess input data to match the model's expected shape."""
    # Ensure input_data is a NumPy array

    categorical_columns=['Dozen', 'Column', 'odd/even', 'red / black', 'series', "Group"]

    # Convert all categorical columns to strings to ensure uniformity
    input_data[categorical_columns] = input_data[categorical_columns].astype(str)
    #Preprocess categorical columns
    input_data = input_data.drop(["Number"], axis=1)
    input_data["Dozen"].replace({"D 1":"D1", "D 2":"D2", "D 3":"D3"}, inplace=True)
    input_data["Column"].replace({"C 1":"C1", "C 2":"C2", "C 3":"C3"}, inplace=True)
    input_data["odd/even"].replace({"nan":"0"}, inplace=True)

    label_encoders = {}
    for column in categorical_columns:
        le = LabelEncoder()
        input_data[column] = le.fit_transform(input_data[column])
        label_encoders[column] = le

    #Scale the data
    scaler = StandardScaler()
    input_data[['Dozen', 'Column', 'Group', 'red / black', 'odd/even']] = scaler.fit_transform(input_data[['Dozen', 'Column', 'Group', 'red / black', 'odd/even']])
    # Fill missing values if necessary
    input_data = input_data.fillna(0)
    ## Converting Data Time Series
    # X = []
    # y=[]
    time_steps = 50
    # data=data.drop(["Number"], axis=1)
    X=input_data.drop(columns=['series']).values
    X= np.array(X)
    # y = np.array(y)
    # print(X.shape)
    X = X.reshape(1, X.shape[0], X.shape[1])
    # input_data = np.array(input_data)

    return X 

# ================================
# Step 3: Perform Inference
# ================================
def predict_class(model, X):
    """Predict class probabilities and labels for the input data."""
    # Get probabilities
    predictions = model.predict(X)

    # Convert probabilities to binary labels (0 or 1)
    # predicted_labels = (probabilities > 0.5).astype(int)
    # print(predicted_labels)
    predicted_labels = np.argmax(predictions).astype(list)
    decoded_labels = label_encoders["series"].inverse_transform([predicted_labels])
    
    return predictions, predicted_labels ,decoded_labels
# ================================
# Step 4: Main Inference Script
# ================================
if __name__ == "__main__":
    # Path to the trained model
    model_path = "lstm_series_model.h5"

    # Load the model
    model = load_trained_model(model_path)

    # Example input data (replace with actual test input)
    # Example assumes time_steps=10, num_features=5
    
    sample_input =data2[50:101]  # Replace with your actual data

    # Preprocess input data
    preprocessed_input = preprocess_input(sample_input)

    # Perform inference
    probabilities, predicted_labels,decoded_labels = predict_class(model, preprocessed_input)

    # Output the results
    print("Predicted Probabilities:", probabilities[-1])
    print("Predicted Labels:", predicted_labels)
    print("Decoded Labels:", decoded_labels[-1])


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
Predicted Probabilities: [0.46145624 0.21465917 0.32388455]
Predicted Labels: 0
Decoded Labels: A


,Number,Dozen,Column,odd/even,red / black,series,Group
101,2,D 1,C 2,EVEN,BLACK,A,G1
102,21,D 2,C 3,ODD,RED,A,G1


In [326]:
data2[:1]

,Number,Dozen,Column,odd/even,red / black,series,Group
0,6,D 1,C 3,EVEN,BLACK,B,G2


In [330]:
model.predict(preprocessed_input[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step


array([[0.6777578]], dtype=float32)

In [388]:
# import numpy as np
# import pandas as pd
# from tensorflow.keras.models import load_model
# from sklearn.preprocessing import StandardScaler

# # ================================
# # Step 1: Load the Trained Model
# # ================================
# def load_trained_model(model_path):
#     """Load the trained model from a file."""
#     model = load_model(model_path)
#     print("Model loaded successfully.")
#     return model

# # ================================
# # Step 2: Preprocess Input Data
# # ================================
# def preprocess_input(input_data, label_encoders, time_steps, num_features):
#     """Preprocess input data to match the model's expected shape."""
#     # Ensure input_data is a DataFrame
#     categorical_columns = ['Dozen', 'Column', 'odd/even', 'red / black', 'series', 'Group']

#     # Convert all categorical columns to strings to ensure uniformity
#     input_data[categorical_columns] = input_data[categorical_columns].astype(str)

#     # Convert all categorical columns to strings to ensure uniformity
#     input_data[categorical_columns] = input_data[categorical_columns].astype(str)
#     #Preprocess categorical columns
#     input_data = input_data.drop(["Number"], axis=1)
#     input_data["Dozen"].replace({"D 1":"D1", "D 2":"D2", "D 3":"D3"}, inplace=True)
#     input_data["Column"].replace({"C 1":"C1", "C 2":"C2", "C 3":"C3"}, inplace=True)
#     input_data["odd/even"].replace({"nan":"0"}, inplace=True)
    
#     # Encoding categorical columns
#     for column in categorical_columns:
#         input_data[column] = label_encoders[column].transform(input_data[column])
    
#     # Normalize 'Dozen', "series" and 'Column' values
#     scaler = StandardScaler()
#     input_data[['Dozen', 'Column', 'odd/even', 'red / black', 'series']] = scaler.fit_transform(input_data[['Dozen', 'Column', 'odd/even', 'red / black', 'series']])

#     # Fill missing values if necessary
#     input_data = input_data.fillna(0)

#     # Converting Data Time Series
#     X = input_data.drop(columns=['Group'])
#     y = input_data['Group']

#     if len(X) < time_steps:
#         padding = pd.DataFrame(np.zeros((time_steps - len(X), num_features)))
#         X = pd.concat([padding, X], ignore_index=True)
#     X = np.array(X.values)
#     y = np.array(y.values)
#     X.reshape(1, time_steps, num_features)

#     return X, y

# # ================================
# # Step 3: Perform Inference
# # ================================
# def predict_class(model, X):
#     """Predict class probabilities and labels for the input data."""
#     # Get probabilities
#     probabilities = model.predict(X)

#     # Convert probabilities to binary labels (0 or 1)
#     predicted_labels = (probabilities > 0.5).astype(int)
#     return probabilities, predicted_labels

# # ================================
# # Step 4: Main Inference Script
# # ================================
# if __name__ == "__main__":
#     # Path to the trained model
#     model_path = "lstm_model.h5"

#     # Load the model
#     model = load_trained_model(model_path)

#     # Example input data (replace with actual test input)
#     sample_input = pd.read_excel("NEW DATA SHEET (copy) with features.xlsx")[:1]  # Replace with your actual data
#     time_steps = 50
#     num_features = 5
#     # Preprocess input data
#     X, y = preprocess_input(sample_input, label_encoders, time_steps, num_features)

#     # Perform inference
#     probabilities, predicted_labels = predict_class(model, X)

#     # Output the results
#     print("Predicted Probabilities:", probabilities)
#     print("Predicted Labels:", predicted_labels)